In [2]:
import sec_parser as sp
from sec_downloader import Downloader
from sec_downloader.types import RequestedFilings

In [2]:
dl = Downloader("student", "jmchoi1993@snu.ac.kr")

In [3]:
reports = dl.get_filing_metadatas(
    RequestedFilings(ticker_or_cik="NVDA", form_type="10-Q", limit=2)
)

In [4]:
html = dl.download_filing(url=reports[0].primary_doc_url).decode()

In [5]:
html

'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<!--XBRL Document Created with the Workiva Platform-->\n<!--Copyright 2024 Workiva-->\n<!--r:a8e8a268-0bb3-4333-933b-c79fd35bc515,g:d448b1d6-0c95-4029-941b-24f65a0744d7,d:1b0efd1e1a3e4e6aa6e902a95be20f04-->\n<html xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:srt="http://fasb.org/srt/2023" xmlns:nvda="http://www.nvidia.com/20240428" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:country="http://xbrl.sec.gov/country/2023" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns="http://www.w3.org/1999/xhtml" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xsi="http://www.w3.org/

In [3]:
# 파일을 읽기 모드로 열기
with open('./amd/amd-20240330.htm', 'r', encoding='utf-8') as file:
    # 파일 내용을 문자열로 읽기
    html_content = file.read()

# 읽은 문자열 출력
print(html_content)


<?xml version='1.0' encoding='ASCII'?>
<!--XBRL Document Created with the Workiva Platform-->
<!--Copyright 2024 Workiva-->
<!--r:3e6f8ac1-ca25-40f9-9b6e-f21410fa49da,g:ce35834c-86c2-4555-a1e0-78646dfc4a12,d:cfae7c6b9321448b8d29819bf0d16901-->
<html xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ecd="http://xbrl.sec.gov/ecd/2024" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns="http://www.w3.org/1999/xhtml" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:us-gaap="http://fasb.org/us-gaap/2024" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:srt="http://fasb.org/srt/2024" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:amd="http://www.amd.com/20240330" xmlns:dei="http://xbrl.sec.gov/dei/2024" xml:lang="en-US"><head><meta http-equ

In [4]:


elements: list = sp.Edgar10QParser().parse(html_content)

demo_output: str = sp.render(elements)
def print_first_n_lines(text: str, *, n: int):
    print("\n".join(text.split("\n")[:n]), "...", sep="\n")
print_first_n_lines(demo_output, n=7)

TopSectionTitle: PART I. FINANCIAL INFORMATION
TopSectionTitle: ITEM 1.CONDENSED CONSOLIDATED FINANCIAL STATEMENTS
TitleElement: Condensed Consolidated Statements of Operations
SupplementaryText: (Unaudited)
TableElement: Table with ~22 rows, ~42 numbers, and 778 characters.
SupplementaryText: See accompanying notes.
TitleElement: Condensed Consolidated Statements of Comprehensive Income (Loss)
...


In [9]:
tree = sp.TreeBuilder().build(elements)

demo_output: str = sp.render(tree)
print(demo_output)

TopSectionTitle: PART I. FINANCIAL INFORMATION
├── TopSectionTitle: ITEM 1.CONDENSED CONSOLIDATED FINANCIAL STATEMENTS
│   ├── TitleElement: Condensed Consolidated Statements of Operations
│   │   ├── SupplementaryText: (Unaudited)
│   │   ├── TableElement: Table with ~22 rows, ~42 numbers, and 778 characters.
│   │   └── SupplementaryText: See accompanying notes.
│   ├── TitleElement: Condensed Consolidated Statements of Comprehensive Income (Loss)
│   │   ├── SupplementaryText: (Unaudited)
│   │   ├── TableElement: Table with ~4 rows, ~8 numbers, and 243 characters.
│   │   └── SupplementaryText: See accompanying notes.
│   ├── TitleElement: Condensed Consolidated Balance Sheets
│   │   ├── SupplementaryText: (Unaudited)
│   │   ├── TableElement: Table with ~39 rows, ~69 numbers, and 1541 characters.
│   │   └── SupplementaryText: See accompanying notes.
│   ├── TitleElement: Condensed Consolidated Statements of Cash Flows
│   │   ├── SupplementaryText: (Unaudited)
│   │   ├── TableE

In [10]:
top_level_sections = [
    item for part in sp.TreeBuilder().build(elements) for item in part.children
]

In [12]:
top_level_section = top_level_sections[0]
levels = sorted(
    {
        k.semantic_element.level
        for k in top_level_section.get_descendants()
        if isinstance(k.semantic_element, sp.TitleElement)
    }
)
level_to_markdown = {level: "#" * (i + 2) for i, level in enumerate(levels)}
level_to_markdown



{0: '##', 1: '###', 2: '####', 3: '#####'}

In [13]:
markdown = ""
markdown += f"# {top_level_section.semantic_element.text}\n"

for node in top_level_section.get_descendants():
    element = node.semantic_element
    if isinstance(element, sp.TextElement):
        markdown += f"{element.text}\n"
    elif isinstance(element, sp.TitleElement):
        markdown += f"{level_to_markdown[element.level]} {element.text}\n"
    elif isinstance(element, sp.TableElement):
        markdown += f"[{element.get_summary()}]"

In [1]:
import re
from io import StringIO
import bs4
import pandas as pd
import sec_parser as sp


def render_view_parsed_export_as(elements):

    title_level_to_markdown = _get_map_title_levels_to_markdown_headings(elements)
    output = []
    for element in elements:
        if isinstance(element, sp.TopSectionTitle):
            output.append(f"# {element.text}")
        elif isinstance(element, sp.TextElement):
            output.append(f"{element.text}")
        elif isinstance(element, sp.TitleElement):
            output.append(f"{title_level_to_markdown[element.level]} {element.text}")
        elif isinstance(element, sp.TableElement):
            unmerged_html: str = _unmerge_cells(element.get_source_code())
            output.append(_pandas_to_markdown(_html_to_pandas(unmerged_html)))

    markdown_text = "\n\n".join(output)
    return markdown_text


def _get_map_title_levels_to_markdown_headings(elements, base_heading_level=2):
    levels = sorted(
        {k.level for k in elements if isinstance(k, sp.TitleElement)},
    )
    adjusted_levels = [level - min(levels) + base_heading_level for level in levels]
    return {
        level: "#" * adjusted_level
        for level, adjusted_level in zip(levels, adjusted_levels)
    }


def _unmerge_cells(html: str) -> str:
    soup = bs4.BeautifulSoup(html, "lxml")
    assert soup is not None, "No table found."
    assert isinstance(soup, bs4.Tag), "Expected a bs4.Tag."
    for td in soup.find_all("td"):
        if td.has_attr("colspan"):
            for _ in range(int(td["colspan"]) - 1):
                new_td = soup.new_tag("td")
                new_td.string = "NaN"
                td.insert_before(new_td)
            td["colspan"] = 1
    return str(soup)


def _pandas_to_markdown(df):
    df = df.fillna("")
    markdown_table = "\n".join(df.to_markdown(index=False).split("\n")[2:])
    markdown_table = re.sub(r" +", " ", markdown_table)
    return markdown_table


def _html_to_pandas(html):
    df = pd.read_html(StringIO(html), flavor="lxml")[0]
    df = df.dropna(how="all")
    df = df.dropna(how="all", axis="columns")
    return df

In [8]:
def save_markdown_to_file(markdown_text, filename="output.md"):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(markdown_text)

In [14]:
import warnings

# 파일을 읽기 모드로 열기
with open('20240312000736.xml', 'r', encoding='utf-8') as file:
    # 파일 내용을 문자열로 읽기
    html_content = file.read()

parser = sp.Edgar10QParser()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Invalid section type for")
    elements1: list = parser.parse(html_content)

/home/ubuntu/softly_ai/Stock-Chatbot/.venv/lib/python3.10/site-packages/sec_parser/utils/bs4_/approx_table_metrics.py:31: UserWarning: Failed to get table metrics: 'NoneType' object has no attribute 'text'
  warnings.warn(f"Failed to get table metrics: {e}", stacklevel=0)


In [15]:
from sec_parser.processing_steps import TopSectionManagerFor10Q, IndividualSemanticElementExtractor, TopSectionTitleCheck

def without_10q_related_steps():
    all_steps = sp.Edgar10QParser().get_default_steps()
    
    # Change 1: Remove the TopSectionManagerFor10Q
    steps_without_top_section_manager = [step for step in all_steps if not isinstance(step, TopSectionManagerFor10Q)]
    
    # Change 2: Replace the IndividualSemanticElementExtractor with a new one that has the top section checks removed
    def get_checks_without_top_section_title_check():
        all_checks = sp.Edgar10QParser().get_default_single_element_checks()
        return [check for check in all_checks if not isinstance(check, TopSectionTitleCheck)]
    return [
        IndividualSemanticElementExtractor(get_checks=get_checks_without_top_section_title_check) 
        if isinstance(step, IndividualSemanticElementExtractor) 
        else step
        for step in steps_without_top_section_manager
    ]

parser2 = sp.Edgar10QParser(get_steps=without_10q_related_steps)
elements2 = parser2.parse(html_content)

/home/ubuntu/softly_ai/Stock-Chatbot/.venv/lib/python3.10/site-packages/sec_parser/utils/bs4_/approx_table_metrics.py:31: UserWarning: Failed to get table metrics: 'NoneType' object has no attribute 'text'
  warnings.warn(f"Failed to get table metrics: {e}", stacklevel=0)


In [16]:
result1 = render_view_parsed_export_as(elements1)
result2 = render_view_parsed_export_as(elements2)
save_markdown_to_file(result1, "dart_test1.md")
save_markdown_to_file(result2, "dart_test2.md")